**1. Импортируем модули и данные (с предварительной обработкой)**

In [87]:
import torch
from torch import nn
import re
import random
import tqdm
import time

In [88]:
!wget https://s3.amazonaws.com/text-datasets/nietzsche.txt

--2024-01-07 10:45:33--  https://s3.amazonaws.com/text-datasets/nietzsche.txt
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.217.10.174, 52.217.113.56, 52.217.84.230, ...
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.217.10.174|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 600901 (587K) [text/plain]
Saving to: ‘nietzsche.txt.2’

nietzsche.txt.2     100%[===================>] 586.82K  --.-KB/s    in 0.1s    

2024-01-07 10:45:34 (3.84 MB/s) - ‘nietzsche.txt.2’ saved [600901/600901]



In [89]:
with open('nietzsche.txt', encoding='utf-8') as f:
    text = f.read().lower()
print('length:', len(text))
text = re.sub('[^a-z ]', ' ', text)
text = re.sub('\s+', ' ', text)

length: 600893


In [90]:
text[:100]

'preface supposing that truth is a woman what then is there not ground for suspecting that all philos'

**2. Создаем словарь**

In [91]:
INDEX_TO_CHAR = sorted(list(set(text)))
CHAR_TO_INDEX = {c: i for i, c in enumerate(INDEX_TO_CHAR)}

In [92]:
len(INDEX_TO_CHAR)

27

In [93]:
print(CHAR_TO_INDEX)

{' ': 0, 'a': 1, 'b': 2, 'c': 3, 'd': 4, 'e': 5, 'f': 6, 'g': 7, 'h': 8, 'i': 9, 'j': 10, 'k': 11, 'l': 12, 'm': 13, 'n': 14, 'o': 15, 'p': 16, 'q': 17, 'r': 18, 's': 19, 't': 20, 'u': 21, 'v': 22, 'w': 23, 'x': 24, 'y': 25, 'z': 26}


**3. Создаем входы и выходы, переводим в цифру**

In [94]:
MAX_LEN = 40
STEP = 3
SENTENCES = []
NEXT_CHARS = []
for i in range(0, len(text) - MAX_LEN, STEP):
    SENTENCES.append(text[i: i + MAX_LEN])
    # созжаем список кусков текста по 40 символов со смещением 3
    NEXT_CHARS.append(text[i + MAX_LEN])
    # создаем список букв, которые следуют за каждым кускос текста, чтобы их предсказывать
print('Num sents:', len(SENTENCES))

Num sents: 193075


In [95]:
SENTENCES[:5] # подается на вход

['preface supposing that truth is a woman ',
 'face supposing that truth is a woman wha',
 'e supposing that truth is a woman what t',
 'upposing that truth is a woman what then',
 'osing that truth is a woman what then is']

In [96]:
NEXT_CHARS[:5] # подается на выход

['w', 't', 'h', ' ', ' ']

In [97]:
# переводим наши вход и выход в цифру и тензор
print('Vectorization...')
X = torch.zeros((len(SENTENCES), MAX_LEN), dtype=int)
Y = torch.zeros((len(SENTENCES)), dtype=int)
for i, sentence in enumerate(SENTENCES):
    for t, char in enumerate(sentence):
        X[i, t] = CHAR_TO_INDEX[char]
    Y[i] = CHAR_TO_INDEX[NEXT_CHARS[i]]

Vectorization...


In [98]:
X.shape

torch.Size([193075, 40])

In [99]:
Y.shape

torch.Size([193075])

In [100]:
# смотрим чо вышло
X[0:1], Y[0]

(tensor([[16, 18,  5,  6,  1,  3,  5,  0, 19, 21, 16, 16, 15, 19,  9, 14,  7,  0,
          20,  8,  1, 20,  0, 20, 18, 21, 20,  8,  0,  9, 19,  0,  1,  0, 23, 15,
          13,  1, 14,  0]]),
 tensor(23))

**4. Создаем загрузчик длиной 512 (батч) для подачи в модель**

In [101]:
BATCH_SIZE=512
dataset = torch.utils.data.TensorDataset(X, Y)
data = torch.utils.data.DataLoader(dataset, BATCH_SIZE, shuffle=True)

In [102]:
for i in data:
  print(i)
  break

[tensor([[20, 15,  0,  ...,  8,  5,  0],
        [20,  5,  5,  ...,  0,  9, 13],
        [21, 19,  9,  ..., 13,  5, 14],
        ...,
        [12, 19,  5,  ...,  0,  4,  9],
        [20,  0,  1,  ...,  5,  0,  8],
        [14,  7, 19,  ...,  6,  5,  3]]), tensor([ 9, 16,  4,  2, 16,  4, 20,  4, 13, 15,  0,  5, 10,  5,  0,  0,  3,  0,
         3,  9, 19, 23, 15, 14,  3, 14,  0, 19, 21,  0, 22,  0, 19,  9,  5,  8,
        12,  8,  9,  1,  0,  2,  2,  9,  0,  0, 23, 19, 15, 20, 18, 25, 15,  5,
        21, 20, 15,  9,  1, 12,  0,  0,  5, 20,  0,  5, 14,  8, 19, 15, 15,  7,
        16, 18, 12,  0, 13,  9, 21,  9,  0,  0, 12,  5, 20, 18,  0,  0, 19,  0,
         0,  0,  0,  2, 16,  7,  9, 14,  0, 20, 19,  5,  0, 15, 14,  5, 18,  4,
        12, 21,  3,  0,  5,  7, 20,  5,  6,  0, 12, 20,  0,  5,  1,  0,  1,  1,
         0,  5, 14, 16,  5,  5, 14,  7, 18, 12,  1, 15, 13, 13,  3, 15,  5,  5,
         5,  4,  0,  6,  4,  6, 18, 19, 21, 15,  9,  0,  1, 23,  1, 12,  5,  0,
        24,  1, 15, 20, 

In [126]:
class NeuralNetwork(nn.Module):
    def __init__(self, rnnClass, dictionary_size, embedding_size, num_hiddens, num_classes):
                    # тип_сетки, длина словаля, эмбеддинг, количество_слоев(циклов?типа), количество классов)
        super().__init__()

        self.num_hiddens = num_hiddens # количество_слоев(циклов?типа)
        self.embedding = nn.Embedding(dictionary_size, embedding_size) # эмбеддинг для каждой буквы словаря, матрица (27,64)
        self.hidden = rnnClass(embedding_size, num_hiddens, batch_first=True) #  модель GRU
        self.output = nn.Linear(num_hiddens, num_classes) # линейный слой

    def forward(self, X):
        out = self.embedding(X) # эмбединг для входа на основе эмбединга словаря штоли
        _, state = self.hidden(out) # скрытое состояние(_) + выход(state)
        predictions = self.output(_[:, -1, :].squeeze())#state[0].squeeze()) # предсказание
        return predictions

In [127]:
model = NeuralNetwork(nn.LSTM, len(CHAR_TO_INDEX), 64, 128, len(CHAR_TO_INDEX))

In [105]:
X.shape

torch.Size([193075, 40])

**5. Модель выдает список уверенности модели в каждом символе, что он следующий**

In [106]:
model(X[0:1])

tensor([-1.0061e-01,  6.6450e-02, -3.7602e-02, -8.7220e-02,  6.0793e-03,
        -3.3691e-01,  6.3828e-02,  8.4020e-02, -6.6685e-02, -4.3806e-02,
         1.7248e-01,  2.8862e-01, -2.8817e-04,  1.5322e-01,  4.6966e-02,
         4.3404e-02, -1.9415e-01, -6.4912e-02,  9.0666e-02,  1.5610e-01,
        -7.0881e-02,  7.7629e-02, -6.3254e-02, -6.6895e-02,  1.4259e-02,
        -8.0954e-02,  7.0748e-03], grad_fn=<ViewBackward0>)

**6. Просто разбираем что и как (отступление)**

**LSTM**

In [93]:
embedding = nn.Embedding(len(INDEX_TO_CHAR), 15)
rnn = nn.LSTM(15,128, batch_first=True)

In [94]:
o, s = rnn(embedding(X[0:10]))
o.shape, o[:, -1, :].squeeze(), len(s), s[0].shape, s[1].shape

(torch.Size([10, 40, 128]),
 tensor([[-0.0187,  0.0633,  0.1252,  ..., -0.0567,  0.0424,  0.0948],
         [-0.1143,  0.0419,  0.1379,  ..., -0.0669,  0.0771,  0.0540],
         [-0.0844,  0.0332,  0.1269,  ..., -0.0330,  0.0328,  0.0432],
         ...,
         [-0.0046,  0.0055,  0.0559,  ..., -0.1016,  0.0666,  0.0771],
         [ 0.0681,  0.0301,  0.0838,  ..., -0.0515, -0.0014,  0.1414],
         [ 0.0604,  0.0300, -0.0110,  ..., -0.0655,  0.0317,  0.0895]],
        grad_fn=<SqueezeBackward0>),
 2,
 torch.Size([1, 10, 128]),
 torch.Size([1, 10, 128]))

**GRU**

In [95]:
rnn = nn.GRU(15,128, batch_first=True)
o, s = rnn(embedding(X[0:10]))
o.shape, len(s), s[0].shape

(torch.Size([10, 40, 128]), 1, torch.Size([10, 128]))

In [96]:
o, s = rnn(embedding(X[0:10]))

In [131]:
model = model.cuda()

**Добавляем вариативность.**
def sample(preds) - функиция которая, имея одинаково высокую вероятность выпадения несколькох букв, дает выпадание то одной, то другой буквы, чтобы не было постоянно одного и того же текста

In [132]:
def sample(preds):
    softmaxed = torch.softmax(preds, 0)
    probas = torch.distributions.multinomial.Multinomial(1, softmaxed).sample()
    return probas.argmax()

def generate_text():
    start_index = random.randint(0, len(text) - MAX_LEN - 1) # случайным образом задаем стартовый индекс из текста

    generated = '' # пустая строка для текста
    sentence = text[start_index: start_index + MAX_LEN] # кусок текста от рэндомного индекса длиной 40
    generated += sentence # добавляем полученный текст в пустую строку

    for i in range(MAX_LEN): # от 0 до 39 включительно
        x_pred = torch.zeros((1, MAX_LEN), dtype=int) # нулевой тензор (1,1,40)
        for t, char in enumerate(generated[-MAX_LEN:]):
            x_pred[0, t] = CHAR_TO_INDEX[char] # пробегаем по нулевому тензору заменяя
                                              # нули на индексы букв согласно словарю
        preds = model(x_pred.cuda()).cpu() # подаем проиндексированный текст в модель
        # preds = model(x_pred.cuda())[0].cpu() это не срабатывало, т.к. на выходе 1 главный элемент, а для sample(preds) нужны все, он сам выберет
        next_char = INDEX_TO_CHAR[sample(preds)] # предсказанный индекс переводим в букву согласно словарю
        generated = generated + next_char # добавляем букву в сроку (пустую, но она не пуста)
    print(generated[:MAX_LEN] + '|' + generated[MAX_LEN:])

In [133]:
generate_text()

or souls into the ultimate intentions of| wfvahegdgfkrdovktc nidvyngwhgtqloyybpbe


In [134]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters())

In [135]:
for ep in range(100):
    start = time.time()
    train_loss = 0.
    train_passed = 0

    model.train()
    for X_b, y_b in data:
        X_b, y_b = X_b.cuda(), y_b.cuda()
        optimizer.zero_grad()
        answers = model(X_b)
        loss = criterion(answers, y_b)
        train_loss += loss.item()

        loss.backward()
        optimizer.step()
        train_passed += 1

    print("Epoch {}. Time: {:.3f}, Train loss: {:.3f}".format(ep, time.time() - start, train_loss / train_passed))
    model.eval()
    generate_text()

Epoch 0. Time: 3.694, Train loss: 2.182
europe it is worst and most varied where|n of way at the hecrerebyraintuecisen ev
Epoch 1. Time: 3.747, Train loss: 1.800
ct in question any kind of cognizance of| is loftrmong the for this no limpoprion
Epoch 2. Time: 3.955, Train loss: 1.658
mber to adopt the same moral or custom t|heythmyoge ehisl good spidinided one to 
Epoch 3. Time: 3.558, Train loss: 1.572
ience in reality there exists between re|spure the ordery has he mattrition of th
Epoch 4. Time: 3.440, Train loss: 1.511
the other old patriot vehemently otherwi|nds others this his be and wascoute thse
Epoch 5. Time: 4.087, Train loss: 1.465
r hypotheses as fully established i have| is with perce conterves of entail the a
Epoch 6. Time: 3.433, Train loss: 1.429
e alone who have devised cause sequence |and bearing uped amothing of it is conte
Epoch 7. Time: 3.572, Train loss: 1.398
or of everything organic inasmuch as all| with the pese tyrancested even all outs
Epoch 8. Time: 3.955, Tr

**Отлично выходит!**

**!!!LSTM можно тоже использовать в данном коде, простой подстановкой, т.к. результат берется из скрытого состояния**

In [ ]:
LSTM действительно училась дольше, но и качество выше, чем у GRU